# Workflow to calculate the quantum efficiency of a polycrystalline material

In [3]:
import subprocess
#import nglview as nv
from ase.calculators.castep import Castep
from ase.atoms import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
import ase.calculators.castep
import ase.io.castep
from ase.io.castep import read_castep_cell

from mp_api.client import MPRester

from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.core.surface import SlabGenerator, generate_all_slabs, Slab, miller_index_from_sites, get_d
from pymatgen.core.interface import Interface
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.wulff import WulffShape
from pymatgen.analysis.structure_matcher import *
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.electronic_structure.bandstructure import BandStructure
from pymatgen.symmetry.kpath import KPathLatimerMunro
from pymatgen.vis.structure_chemview import quick_view

#from wulffpack import SingleCrystal

from python_functions import *
from calculation_class import *
#from ../../script_share/bs_plotter_felix_mod import *

In [ ]:
view(read('Cu_111_defective.xyz'))

In [ ]:
pip install mp-api

I recommend using pymatgen to generate the bulk/slab and then convert it to an ASE atom object. The advantages of this approach are:
- pymatgen has a better bulk/slab generation algorithm 
- castep can link a calculator to the Atom object


#### Download and save .cif file for necessary materials

In [ ]:
import json
api_key = ''
with MPRester(api_key) as m:
    #results = m.get_dos_by_material_id("mp-30")
    results = m.summary.search(material_ids=["mp-124"])
    list_of_available_fields = m.summary.available_fields
#print(list_of_available_fields)
#print(results[0].structure)
Ag_primitive_bulk = results[0].structure
Ag_primitive_bulk.to(filename="./structures/data_files/materials/Ag_primitive.cif")
print(Ag_primitive_bulk)
print(type(Ag_primitive_bulk))
# res_json = results.as_dict()
# with open('./structures/jsons/materials/.json', 'w') as f:
#     json.dump(results.as_dict(), f)

In [ ]:
ag = read_cell2pmg('./structures/Graphene_super.cell')
view(AseAtomsAdaptor.get_atoms(ag))

In [ ]:
gaas = AseAtomsAdaptor.get_structure(read('./GaAs_conv_std.cif'))
gaas.scale_lattice(5.653**3)
print(gaas)
gaas_prim = SpacegroupAnalyzer(gaas).find_primitive()
print(gaas_prim)


In [ ]:
seed = 'Ag100_new19e_relax'
#structure = Structure.from_file("./structures/data_files/materials/Ag_primitive.cif")
#bulk = Structure.from_file(filename="Cu_metal_fcc.cif")

In [ ]:
structure = AseAtomsAdaptor.get_structure(read('./structures/Ag_bulk_eos/Ag_bulk_eos.xyz',index = -1))
structure.to(filename = "./structures/data_files/materials/Ag_bulk_minimum_eos.cif")

In [ ]:
mg_o = AseAtomsAdaptor.get_structure(read('./structures/MgO_bulk_eos/felix/MgO_bulk_eos.xyz', index=-1))
mg_o_prim = SpacegroupAnalyzer(mg_o).get_primitive_standard_structure()
mg_o_conv = SpacegroupAnalyzer(mg_o).get_conventional_standard_structure()
mgo_100 = SlabGenerator(mg_o_conv,[1,0,0],5,15,max_normal_search=90,center_slab=True).get_slab()

In [ ]:
view(AseAtomsAdaptor.get_atoms(mgo_100))

In [2]:
cu_prim = Structure.from_file('./Cu_metal_fcc.cif')
cu_conv = SpacegroupAnalyzer(cu_prim).get_conventional_standard_structure()
cu_100 = SlabGenerator(cu_conv,[1,0,0],18,15,max_normal_search=90,center_slab=True).get_slab()

In [ ]:
ag = read_cell2pmg('./structures/Ag_convergence/Ag_bulk.cell')
ag_conv = SpacegroupAnalyzer(ag).get_conventional_standard_structure()
ag_prim = SpacegroupAnalyzer(ag).get_primitive_standard_structure()
ag_100 = SlabGenerator(ag_conv,[1,0,0],18,15,max_normal_search=90,center_slab = True).get_slab()
#view(AseAtomsAdaptor.get_atoms(ag_100))

In [ ]:
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer

sub_analyzer = SubstrateAnalyzer()
matches = list(sub_analyzer.calculate(film=mg_o_prim,substrate=ag_prim,film_millers=[(1,0,0)],substrate_millers=[(1,0,0)]))
len(matches)

FileNotFoundError: [Errno 2] No such file or directory: './structures/Ag_convergence/Ag_bulk.cell'

In [ ]:
matches[1].as_dict()

In [ ]:
help(matches[0])
matches

In [ ]:
view(AseAtomsAdaptor.get_atoms(matches[-1]))

In [ ]:
help(SubstrateAnalyzer)

In [ ]:
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.zsl import ZSLGenerator

zsl = ZSLGenerator()
cib = CoherentInterfaceBuilder(film_structure=mg_o_conv,
                               substrate_structure=ag_conv,
                               film_miller=(1,0,0),
                               substrate_miller=(1,0,0),
                               zslgen=zsl)

In [ ]:
cib.terminations

In [ ]:
for match in cib.zsl_matches:
    print(match.match_area)

In [ ]:
cib.zsl_matches[0].match_transformation

In [ ]:
interfaces = list(cib.get_interfaces(termination=cib.terminations[0], substrate_thickness=10, film_thickness=1, vacuum_over_film=20))
len(interfaces)

In [ ]:
interface_2 = interfaces[-1]
interface_2.translate_sites(range(len(interface_2)),[0,0,0])
view(AseAtomsAdaptor.get_atoms(interface_2))
interface_2

In [ ]:
interface = interfaces[0]
interface.translate_sites(range(len(interface)),[0,0,0]) 
interface

In [ ]:
view(AseAtomsAdaptor.get_atoms(interface))

In [ ]:
victor = read_cell2pmg('./structures/AgMgO_interface/AgMgO100.cell')
view(AseAtomsAdaptor.get_atoms(victor))

In [ ]:
help(CoherentInterfaceBuilder)

In [ ]:
matcher = StructureMatcher(attempt_supercell=True)

In [ ]:
view(AseAtomsAdaptor.get_atoms(cu_100))

In [ ]:
layered_structure = AseAtomsAdaptor.get_atoms(ag_100) + AseAtomsAdaptor.get_atoms(mgo_100)
view(layered_structure)

In [ ]:
view(read('./MgO_Ag100_interface.xyz'))

In [ ]:
#primitive_cell = Structure.from_file("./structures/data_files/materials/Ag_bulk_minimum_eos.cif")
primitive_cell = Structure.from_file("./structures/data_files/materials/Ag_primitive.cif")
primitive_cell.scale_lattice(16.6283)
print(primitive_cell)
conv_cell = SpacegroupAnalyzer(primitive_cell).get_conventional_standard_structure()
#conv_cell

In [1]:
from ase.build import bulk

a1 = bulk('Cu', 'fcc', a=3.6)

a2 = bulk('Cu', 'fcc', a=3.6, orthorhombic=True)

a3 = bulk('Cu', 'fcc', a=3.6, cubic=True)

In [5]:
a1

Atoms(symbols='Cu', pbc=True, cell=[[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

In [3]:
structure = SlabGenerator(conv_cell,[1,0,0],18,20,max_normal_search=90,center_slab=True).get_slab()
print(structure)

Slab Summary (Ag10)
Reduced Formula: Ag
Miller index: (1, 0, 0)
Shift: 0.0000, Scale Factor: [[0 1 0]
 [0 0 1]
 [1 0 0]]
abc   :   2.864976   2.864976  40.516876
angles:  90.000000  90.000000  90.000000
Sites (10)
1 Ag     1.000000     0.000000     0.725000
2 Ag     0.500000     0.500000     0.675000
3 Ag     0.000000     0.000000     0.625000
4 Ag     0.500000     0.500000     0.575000
5 Ag     0.000000     1.000000     0.525000
6 Ag     0.500000     0.500000     0.475000
7 Ag     0.000000     0.000000     0.425000
8 Ag     0.500000     0.500000     0.375000
9 Ag     0.000000     1.000000     0.325000
10 Ag     0.500000     0.500000     0.275000


## Build Cu Supercells

In [3]:
seed = 'Cu100_od_rerun'

In [ ]:
filename = "./structures/AgMgO_interface/AgMgO100.cell"
agmgo_int = read_cell2pmg(filename)
agmgo_int

## Calculation Setup

In [ ]:
seed = 'AgMgO_interface_run'

In [ ]:
GeneralOptions = {
    'seed' : seed,
    'directory': f"./structures/{seed}",
    'tasks' : ['Geometry','Spectral','OD_Fermi','Workfunction','OD_Photo'],
    #'tasks' : ['Geometry','Spectral','OD_Fermi','Workfunction','OD_Photo','BandStructure'],
    'template_scripts' : True
}

In [8]:
CastepOptions = {
    'xc_functional': 'PBEsol',
    'energy_cutoff': 825,
    'elec_energy_tol': 1e-8,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'metals_method' : 'DM', # Default - EDFT; if FIX_OCCUPANCY : FALSE, the default is DM
    'mixing_scheme' : 'Broydon', #Choose from Broydon or Pulay - Default: Broydon
    'smearing_width' : 300,
    'extra_bands': True,
    'percent_extra_bands' : 80,
    'max_scf_cycles': 1000,
    'spin_polarized': False,
    'write_potential': True,
    'write_density': True,
    'spectral_task' : 'All', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All
    'continuation': False,
    #Cell File Instructions
    'kpoints': (16,16,1),
    'offset_mp_grid': False,
    'kpoint_mp_offset':('1/32','1/32','0'),
    'spectral_kpt_grid': (16,16,16),
    'generate_symmetry' : False,
    'snap_to_symmetry': False,
    'fix_all_cell': True,
    'calculate_pdos': True,
    'fine_grid_scale' : 3.0,
    #'bandstruct_path': 'WLGXWK',
    'bandstruct_path': 'XGMX',
    'bandstruct_kpt_dist': 0.1,   
}

In [ ]:
OptaDosOptions = {
    'optados_task': 'all', # Choose: dos(default), compare_dos, compare_jdos, jdos, pdos, optics, core, all
    'broadening': 'adaptive', #Choose: adaptive(default), fixed, linear
    'iprint': '1', #Choose: 1 - bare minimum, 2 - with progress reports, 3 - fulld debug output
    'efermi': 'optados', #Choose: optados - recalculate, file - read from CASTEP file, insulator - count filled bands, float - supplied by user
    'dos_spacing': '0.001', #DOS spacing in unit (default: eV): default - 0.1
    'pdos': 'angular', #Choose: angular, species_ang, species, sites or more detailed descriptions such as: 
    #PDOS : sum:Si1-2(s) - sum of s-chnnls on 2 Si atms (1 proj), 
    #PDOS : Si1;Si2(s) - DOS on Si atom 1 and DOS on s-channel of Si atom 2 (2 proj) 
    'photo_options': {
        'jdos_spacing' : 0.1,
        'jdos_max_energy' : 25,
        'broadening' : 'linear',
        'linear_smearing' : 0.026,
        'optics_geom' : 'unpolar',
        'optics_intraband' : True,
        'optics_qdir' : [1, 1.000, 1.0000],
        'hybrid_linear' : True,
        'photo_photon_energy' : 4.7,
        'photo_work_function' : 4.556,
        'photo_surface_area' : 6.339744873,
        'photo_slab_volume' : 190.942338,
        'photo_elec_field' : 0,
        'photo_imfp_const' : 6.13,
        'photo_model' : ['1step','3step'],
        'photo_momentum' : 'crystal',
        'photo_temperature' : 300,
        'photo_theta_lower' : 59,
        'photo_theta_upper' : 61,
        'iprint' : 1,
        },
    'sweep_options': {
        'parameter' : 'photon_energy', # choose photon_energy or temp or elec_field or work_function
        'min' : 4.0,
        'max' : 6.2,
        'stepsize' : 0.1,
        },
}

In [ ]:
cu100_calc = PhotoemissionCalculation(cu_100, GeneralOptions, CastepOptions,OptaDosOptions)
generate_input_files(**cu100_calc.options)

#### Generate the surface structure

In [ ]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
print(bulk)
#view(AseAtomsAdaptor.get_atoms(bulk))
analyser = SpacegroupAnalyzer(bulk)
bulk_conv = analyser.get_refined_structure()
#view(AseAtomsAdaptor().get_atoms(bulk_conv))
bulk_ase = AseAtomsAdaptor().get_atoms(bulk_conv)

#print(bulk)
#surface = ase.build.surface(lattice = bulk_ase, indices = (1,0,0), layers = 8, vacuum=15, tol=1e-10, periodic=True)
#view(surface)b

In [ ]:
#view(read('./structures/Cu_surf_111/Cu_surf_111.xyz'))
view(AseAtomsAdaptor.get_atoms(read_cell2pmg('./structures/geometry_opt/Cu_surf_111/Cu_surf_111.cell')))

In [ ]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
seed = 'Cu_bulk_murn_ext'
murn, minimal_struct, energies,volumes = read_murnaghan_outputs(seed = seed, structure = bulk,path = f'./structures/EOS_tests/PBEsol/{seed}/')
fig= murn.plot()
plt.style.use('seaborn-darkgrid')

conventional = SpacegroupAnalyzer(minimal_struct).get_conventional_standard_structure()
#view(AseAtomsAdaptor.get_atoms(conventional))

In [ ]:
data_111 = SlabGenerator(conventional,[1,1,1],31,20,center_slab=True).get_slabs()[0].get_orthogonal_c_slab()
data_111_2 = SlabGenerator(conventional,[1,1,1],31,20,center_slab=True,primitive=False).get_slabs()[0].get_orthogonal_c_slab()
data_110 = SlabGenerator(conventional,[1,1,0],18,20,center_slab=True).get_slabs()[0]
data_100 = SlabGenerator(conventional,[1,0,0],26,20,center_slab=True).get_slabs()[0]
view(AseAtomsAdaptor.get_atoms(data_111))
view(AseAtomsAdaptor.get_atoms(data_111_2))

In [ ]:
from pymatgen.transformations.standard_transformations import *
rotation = RotationTransformation([0,0,1], 120)
rotated = rotation.apply_transformation(data_111)
print(rotated)
rotated_2 = AseAtomsAdaptor.get_atoms(data_111)
rotated_2.rotate(120,'z', rotate_cell=True)
view(rotated_2)
#view(AseAtomsAdaptor.get_atoms(rotated))

In [ ]:
view(read('./structures/Cu_surf_111/Cu_surf_111.xyz'))

In [ ]:
#surfaces = generate_all_slabs(structure=bulk_conv,max_index=1,min_slab_size=16, min_vacuum_size=6, center_slab=True, in_unit_planes=True)
surfaces = {}

surfaces['111'] = SlabGenerator(bulk_conv, [1,1,1],  min_slab_size=32, min_vacuum_size=22, center_slab = True).get_slabs()[0]
surfaces['110'] = SlabGenerator(bulk_conv, [1,1,0],  min_slab_size=18, min_vacuum_size=22, center_slab = True).get_slabs()[0]
surfaces['100'] = SlabGenerator(bulk_conv, [1,0,0],  min_slab_size=26, min_vacuum_size=22, center_slab = True).get_slabs()[0]
# print(surfaces)
for index,item in enumerate(surfaces.keys()):
    #print(surfaces)
    #print(len(surfaces[item]))
    #print(miller_index_from_sites(surfaces[item].lattice, surfaces[item].frac_coords, False))
    #print(surfaces[item])
    #write(f'./structures/xyz/Cu_surf_{hkl[index]}.xyz',AseAtomsAdaptor.get_atoms(surfaces[item]))
    view(AseAtomsAdaptor.get_atoms(surfaces[item]))

In [ ]:
surface_111_ortho = SlabGenerator(bulk, [1,1,0], min_slab_size=16, min_vacuum_size=6, center_slab = True, in_unit_planes=True).get_slabs()[0].get_orthogonal_c_slab()
surface_111 = SlabGenerator(bulk, [1,1,0], min_slab_size=16, min_vacuum_size=6, center_slab = True, in_unit_planes=True).get_slabs()[0]
surface_111_ase = ase.build.surface(lattice = bulk_ase, indices = [1,1,0], layers = 16, vacuum=15, tol=1e-10, periodic=True)
#view(AseAtomsAdaptor.get_atoms(surface_111_ortho))
print(surface_111_ortho)
#print(f'Pymatgen d = {get_d(surface_111)}')
#a = np.rad2deg(np.arcsin(get_d(surface_111_ortho)/get_d(surface_111)))
#print(a)
#print(f'Pymatgen adjusted d = {get_d(surface_111)*np.sin(np.deg2rad(60))}')
#print(f'Pymatgen ortho d = {get_d(surface_111_ortho)}')
#print(f'ASE d = {get_d(AseAtomsAdaptor.get_structure(surface_111_ase))}')

In [ ]:
surfaces = generate_all_slabs(structure=bulk_conv,max_index=1,min_slab_size=16, min_vacuum_size=6, center_slab=True, in_unit_planes=True)
indices = [[1,1,0]]#,[1,0,0]]#,[1,1,0]]
hkl = ['110','100','110']
surfaces = {}
for index,item in enumerate(indices):
    surfaces[hkl[index]] = ase.build.surface(lattice = bulk_ase, indices = item, layers = 16, vacuum=15, tol=1e-10, periodic=True)
for item in surfaces.keys():
    view(surfaces[item])

In [ ]:
surfaces = {}
data_111 = read('./structures/Cu_surf_111/Cu111_rotated.xyz')
# data_111 = SlabGenerator(conventional,[1,1,1],31,20,center_slab=True).get_slabs()[0].get_orthogonal_c_slab()
# data_110 = read('./structures/Cu_surf_110/Cu_surf_110.xyz')
# data_100 = read('./structures/Cu_surf_100/Cu_surf_100.xyz')
surfaces['111'] = {'structure' : data_111}
#surfaces['110'] = {'structure' : data_110}
#surfaces['100'] = {'structure' : data_100}
# surfaces['111']['bandpath'] = 'MGKM' #111 MGKM hex primitive
# surfaces['110']['bandpath'] = "GYXSGX" #110  GYXSGX primitive orthorhombic"
# surfaces['100']['bandpath'] = 'XGMX' #100 XGMX tetragonal primitive

GeneralOptions = {
    'directory': f"./structures/",
    'seed' : seed,
    'tasks' : [],
}
CastepOptions  = {
    'directory': f"./structures/",
    'seed_name': seed,
    # Param File Instructions
    'task': 'GeometryOptimization', #Choose: SinglePoint, BandStructure, Spectral, GeometryOptimization
    'spectral_task' : 'All', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
    'calculate_pdos': True,
    'xc_functional': 'PBEsol',
    'energy_cutoff': 600,
    'elec_energy_tol': 1e-8,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'mixing_scheme' : 'Pulay', #Choose from Broydon or Pulay
    'smearing_width' : 300, # Temperature in Kelvin
    'spin_polarized': False,
    'max_scf_cycles': 1000,
    'write_potential': True,
    'write_density': True,
    'extra_bands': True,
    #Cell File Instructions
    'kpoints': (32,32,1),
    'snap_to_symmetry': True,
    'generate_symmetry': True,
    'fix_all_cell': True,
    'continuation': False,
    'bandstruct_path': 'GXWKGLUWLK',
    'bandstruct_kpt_dist': 0.0184,
    'spectral_kpt_grid': (32,32,1)    
}
PBSOptions = {
    'seed_name': '',
    #choose from castep_19, castep_18, castep_18_mod, optados
    #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
    #'tasks_seeds' : [[castep_opt_template['task'],seed, 'castep_18']],
    #'tasks_seeds': [['Spectral', seed,'castep_18'],['Bandstructure', seed,'castep_18'],['OptaDOS', seed,'optados'],['Spectral',seed,'castep_18_mod']], 
    'tasks_seeds': [['SinglePoint', seed,'castep_18']],
    'nodes' : 1,
    'cpus' : 64,
    'memory' : 100,
    'walltime' : '06:00:00'
}
OptaDOSOptions = {
    'seed_name': seed,
    'optados_task': 'all', # Choose: dos(default), compare_dos, compare_jdos, jdos, pdos, optics, core, all, photoemission
    'broadening': 'adaptive', #Choose: adaptive(default), fixed, linear
    'iprint': '1', #Choose: 1 - bare minimum, 2 - with progress reports, 3 - fulld debug output
    'efermi': 'optados', #Choose: optados - recalculate, file - read from CASTEP file, insulator - count filled bands, float - supplied by user
    'dos_spacing': '0.001', #DOS spacing in unit (default: eV): default - 0.1
    'pdos': 'angular', #Choose: angular, species_ang, species, sites or more detailed descriptions such as: 
    #PDOS : sum:Si1-2(s) - sum of s-chnnls on 2 Si atms (1 proj), 
    #PDOS : Si1;Si2(s) - DOS on Si atom 1 and DOS on s-channel of Si atom 2 (2 proj) 
    'photo_options' : {
        'work_function' : 4.556,
        'surface_area' : 6.339744873,
        'slab_volume' : 190.942338,
        'elec_field' : 0,
        'imfp_const' : 19.0,
        'JDOS_SPACING' : 0.1,
        'JDOS_MAX_ENERGY' : 25,
        'BROADENING' : 'linear',
        'OPTICS_GEOM' : 'unpolar',
        'optics_qdir' : [1, 1, 1],
        'photon_energy' : 21.2,
        'linear_smearing' : 0.026,
        'fixed_smearing' :  0.026,
        'optics_intraband' : True,
        'photo_output' : 'be',
        'photo_model' : '1step',
        'momentum' : 'crystal',
        'hybrid_linear' : True,
        'temp' : 300,
        'theta_lower' : 59,
        'theta_upper' : 61
    }
}

for item in surfaces.keys():
    seed = f'Cu_surf_{item}'

    CastepOptions['directory'] = f"./structures/{seed}" 
    PBSOptions['seed'] = seed
    OptaDOSOptions['seed'] = seed
    options = {'castep':CastepOptions,'pbs':PBSOptions,'optados':OptaDOSOptions}
    #castep_opt_template['bandstruct_path'] = surfaces[item]['bandpath']
    
    generate_castep_input(calc_struct=surfaces[item]['structure'], **options)
    #generate_optados_input(**options)
    generate_qsub_file(**options)

#### Run the calculations

In [ ]:
#folder_change_command = 'cd {}'.format(seed)
#submit_command = 'qsub {}.qsub'.format(seed)

#subprocess.call(folder_change_command)
#subprocess.call(submit_command)
#subprocess.call('cd ..')

#### Extract the final energies

In [ ]:
bulk_out = CastepOutput('./structures/Cu_bulk_bandstruct/Cu_bulk_bandstruct.castep')
surface_out = CastepOutput('./structures/Cu_surf_111_photo/Cu_surf_111.castep')

#### Calculate the surface energy

In [ ]:
print(surface_out.number_atoms)
print(surface_out.structure.volume/surface_out.structure.lattice.c)
surf_energy_111 = calc_surface_energy(bulk_out.ks_total_energy, surface_out.ks_total_energy,surface_out.number_atoms, surface_out.structure.volume/surface_out.structure.lattice.c)
factor = 16.02176565
print(f'The surface energy of the Cu[111] surface is: {surf_energy_111} eV/Angstrom^2.\nThis is equal to: {surf_energy_111*factor} J/m^2')


#### Get the % of each surface from a  Wulff construction

In [ ]:
#mat_structure = bulk
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
# facets_energies = {
#     (1,0,0) : 0.1161,
#     (1,1,0) : 0.1211,
#     (1,1,1) : 0.1055
# } #energies in eV/Angstrom^2
facets_energies = {
    (1,0,0) : 1.551,
    (1,1,0) : 1.736,
    (1,1,1) : 1.335
} #energies in J/m^2

different_fractions = get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)

# Testing of performance by pymatgen wulff shape class
# Results: the ratios of the facets is different than expected and seems incorrect with the wulffshape class

# facets = [(1,0,0),(1,1,0),(1,1,1)]
# energies = [1.86,1.94,1.69]
# def new_wulff_fractions(mat_structure, indices,facet_energies):
#     lattice = SpacegroupAnalyzer(mat_structure).find_primitive()
#     print(lattice)
#     shape = WulffShape(lattice.lattice, indices, facet_energies)
#     return shape.area_fraction_dict
# facets_fractions =  get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)
# different_fractions = get_wulff_fractions(AseAtomsAdaptor.get_atoms(bulk),facets_energies)
# print(facets_fractions)
print(different_fractions)

#### Launch OptaDOS

#### Get QE from OptaDOS output

#### Calculate the weighted average QE

#### Calculate and display plenty more properties (DOS, bands, ...)